In [1]:
!pip install --index-url https://download.pytorch.org/whl/cu126 \
torch

Looking in indexes: https://download.pytorch.org/whl/cu126


In [2]:
# ==========================================================
# Quantformer: per-ticker time split + weighted training + per-ticker metrics
# ==========================================================
import os, pickle, numpy as np
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

In [5]:
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [6]:
# ---------- Load preprocessed dataset ----------
load_path = "/content/gdrive/MyDrive/sp500_quantformer_preprocessed.pkl"  # <-- change if needed
with open(load_path, "rb") as f:
    data_bundle = pickle.load(f)

X_all = data_bundle["X_all"]                 # shape (N, seq_len, F)
y_all = data_bundle["y_all"]                 # continuous next-period returns (N,)
sample_tickers = data_bundle["sample_tickers"]  # per-sample ticker strings (N,)
mu = data_bundle.get("mu", None)             # optional (1, seq_len, F)
sd = data_bundle.get("sd", None)

print("✅ Loaded:", X_all.shape, y_all.shape, sample_tickers.shape)

✅ Loaded: (588886, 60, 1) (588886,) (588886,)


In [7]:
# ---------- Device ----------
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)

# ---------- Labels: 3 equal bins on continuous returns ----------
def label_three_bins(y):
    y = np.asarray(y, dtype=float).reshape(-1)
    q = np.quantile(y, [1/3, 2/3])
    return np.digitize(y, q, right=True).astype(np.int64)

y_cls = label_three_bins(y_all)
num_bins = int(y_cls.max()) + 1
print("Class balance:", {int(k): int(v) for k, v in zip(*np.unique(y_cls, return_counts=True))})

# ---------- Normalize (dataset-level) ----------
X_used = X_all.astype(np.float32)
if mu is not None and sd is not None:
    X_used = (X_used - mu) / (sd + 1e-9)
else:
    mu = X_used.mean(axis=0, keepdims=True)
    sd = X_used.std(axis=0, keepdims=True) + 1e-9
    X_used = (X_used - mu) / sd

assert np.isfinite(X_used).all(), "NaN/Inf in normalized features"

Device: cuda
Class balance: {0: 196296, 1: 196295, 2: 196295}


In [8]:
# ---------- Per-ticker chronological split (no leakage; each ticker contributes to val) ----------
VAL_FRAC = 0.20
idx_all  = np.arange(len(X_used))
tick_all = np.asarray(sample_tickers)

train_idx, val_idx = [], []
for tkr in np.unique(tick_all):
    m = (tick_all == tkr)
    idx_t = idx_all[m]                 # chronological order by construction
    n_t = len(idx_t)
    if n_t < 5:                        # tiny series -> all in train
        train_idx.extend(idx_t.tolist())
        continue
    cut = int((1.0 - VAL_FRAC) * n_t)
    train_idx.extend(idx_t[:cut].tolist())
    val_idx.extend(idx_t[cut:].tolist())

train_idx = np.array(train_idx); train_idx.sort()
val_idx   = np.array(val_idx);   val_idx.sort()

X_tr, X_va = X_used[train_idx], X_used[val_idx]
y_tr, y_va = y_cls[train_idx],  y_cls[val_idx]
tickers_tr, tickers_va = tick_all[train_idx], tick_all[val_idx]
y_va_cont = y_all[val_idx]  # continuous targets aligned to val

print(f"Train samples: {len(y_tr)} | Val samples: {len(y_va)}")

# ---------- Focus tickers & per-sample weights ----------
focus_tickers = ["TSLA", "AAPL", "NVDA"]  # edit this list
focus_tickers = [t.upper().strip() for t in focus_tickers]
multiplier = 8.0                           # try 5–10

weights = np.ones(len(y_tr), dtype=np.float32)
for t in focus_tickers:
    m = (np.char.upper(np.char.strip(tickers_tr.astype(str))) == t)
    if m.any():
        weights[m] *= multiplier
weights /= weights.mean()

print("Mean train weight:", float(weights.mean()))
for t in focus_tickers:
    m = (np.char.upper(np.char.strip(tickers_tr.astype(str))) == t)
    print(f"{t}: train samples={int(m.sum())}, mean weight={float(weights[m].mean()) if m.any() else 'NA'}")

Train samples: 471104 | Val samples: 117782
Mean train weight: 0.9999995827674866
TSLA: train samples=956, mean weight=7.6730170249938965
AAPL: train samples=956, mean weight=7.6730170249938965
NVDA: train samples=956, mean weight=7.6730170249938965


In [9]:
# ---------- DataLoaders ----------
X_tr_t = torch.tensor(X_tr, dtype=torch.float32)
y_tr_t = torch.tensor(y_tr, dtype=torch.long)
w_tr_t = torch.tensor(weights, dtype=torch.float32)

X_va_t = torch.tensor(X_va, dtype=torch.float32)
y_va_t = torch.tensor(y_va, dtype=torch.long)

train_loader = DataLoader(TensorDataset(X_tr_t, y_tr_t, w_tr_t), batch_size=512, shuffle=True)
val_loader   = DataLoader(TensorDataset(X_va_t, y_va_t), batch_size=1024, shuffle=False)

# ---------- Model ----------
class Quantformer(nn.Module):
    def __init__(self, in_dim, num_bins, d_model=128, nhead=8, num_layers=4, dim_ff=256, dropout=0.2):
        super().__init__()
        self.embed = nn.Linear(in_dim, d_model)
        layer = nn.TransformerEncoderLayer(
            d_model=d_model, nhead=nhead, dim_feedforward=dim_ff,
            batch_first=True, norm_first=True, dropout=dropout, activation="gelu"
        )
        self.encoder = nn.TransformerEncoder(layer, num_layers)
        self.head = nn.Sequential(nn.LayerNorm(d_model), nn.Linear(d_model, num_bins))
    def forward(self, x):
        z = self.encoder(self.embed(x))   # (B, T, d)
        return self.head(z.mean(1))       # mean-pool over time

FEATS = X_used.shape[2]
#model = Quantformer(in_dim=FEATS, num_bins=num_bins).to(device)
model = Quantformer(
    in_dim=FEATS, num_bins=num_bins,
    d_model=192, nhead=12, num_layers=6,
    dim_ff=384, dropout=0.25
).to(device)

/usr/local/lib/python3.12/dist-packages/torch/nn/modules/transformer.py:392: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.norm_first was True
  warnings.warn(


In [10]:
# ---------- Loss, Optimizer, Scheduler ----------
counts = np.bincount(y_tr, minlength=num_bins).astype(np.float32)
imbalance = (counts.min()/counts.max()) < 0.7
if imbalance:
    w_cls = counts.sum() / np.clip(counts, 1, None)
    class_w = torch.tensor(w_cls / w_cls.mean(), dtype=torch.float32, device=device)
    crit_base = nn.CrossEntropyLoss(weight=class_w, label_smoothing=0.03)
else:
    crit_base = nn.CrossEntropyLoss(label_smoothing=0.03)

def weighted_ce(logits, targets, sample_w):
    return (crit_base(logits, targets) * sample_w).mean()

opt   = torch.optim.AdamW(model.parameters(), lr=3e-4, weight_decay=1e-4)
sched = torch.optim.lr_scheduler.CosineAnnealingLR(opt, T_max=40)

In [11]:
# ---------- Training + Metrics ----------
best = (1e9, None); patience=6; bad=0
best_path = "/content/gdrive/MyDrive/quantformer_best.pth"

# IC helper (optional)
try:
    from scipy.stats import spearmanr
    def spearman_ic(a, b):
        return spearmanr(a, b).correlation
except Exception:
    spearmanr = None
    def spearman_ic(a, b):
        return float("nan")

In [12]:
for ep in range(1, 61):  # up to 60 epochs; early stopping will cut earlier
    model.train(); tr_loss = 0.0
    for xb, yb, wb in train_loader:
        xb, yb, wb = xb.to(device), yb.to(device), wb.to(device)
        opt.zero_grad(set_to_none=True)
        logits = model(xb)
        loss = weighted_ce(logits, yb, wb)
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        opt.step()
        tr_loss += loss.item() * xb.size(0)
    tr_loss /= len(train_loader.dataset)

    # ---- Validation ----
    model.eval(); va_loss=0.0; correct=0; total=0
    all_probs = []
    with torch.no_grad():
        for xb, yb in val_loader:
            xb, yb = xb.to(device), yb.to(device)
            logits = model(xb)
            p = torch.softmax(logits, 1)
            va_loss += crit_base(logits, yb).item() * xb.size(0)
            correct += (p.argmax(1) == yb).sum().item()
            total += yb.numel()
            all_probs.append(p.cpu().numpy())
    va_loss /= len(val_loader.dataset)
    va_acc  = correct / total
    print(f"Epoch {ep:02d} | train {tr_loss:.4f} | val {va_loss:.4f} | val-acc {va_acc:.3f}")
    sched.step()

    # ---- Global metrics: IC + Precision@10% ----
    all_probs = np.concatenate(all_probs, axis=0)    # (Nv, 3)
    preds_va  = all_probs.argmax(1)
    top_score = all_probs[:, 0]

    # Align safely
    Nv = len(top_score)
    y_va_cont_aligned = y_va_cont[:Nv]
    y_va_cls_aligned  = y_va[:Nv]
    tickers_va_aligned = tickers_va[:Nv]

    ic = spearman_ic(top_score, y_va_cont_aligned)
    K = max(1, int(0.10 * Nv))
    idx_top = np.argsort(-top_score)[:K]
    prec_at_k = (y_va_cont_aligned[idx_top] > 0).mean()
    print(f"   · Val IC: {ic:.4f} | Precision@10%: {prec_at_k:.3f}")

    # ---- Per-ticker metrics (val-acc + P@10%) for focus tickers ----
    tickers_va_norm = np.array([str(t).upper().strip() for t in tickers_va_aligned])
    try:
        import pandas as pd
        cnt = pd.Series(tickers_va_norm).value_counts()
        present = {t: int(cnt.get(t, 0)) for t in focus_tickers}
        print("   · focus val counts:", present)
    except Exception:
        pass

    for t in focus_tickers:
        m = (tickers_va_norm == t)
        n_t = int(m.sum())
        if n_t == 0:
            print(f"   · {t} val-acc: NA | P@10%: NA (n=0)")
            continue
        acc_t = (preds_va[m] == y_va_cls_aligned[m]).mean()
        Kt = max(1, int(0.10 * n_t))
        idx_local = np.argsort(-top_score[m])[:Kt]
        prec_t = (y_va_cont_aligned[m][idx_local] > 0).mean()
        print(f"   · {t} val-acc: {acc_t:.3f} | P@10%: {prec_t:.3f} (n={n_t})")

    # ---- Early stopping on val loss ----
    if va_loss < best[0] - 1e-4:
        best = (va_loss, {k: v.detach().cpu().clone() for k, v in model.state_dict().items()})
        bad = 0
        torch.save({"state_dict": best[1], "mu": mu, "sd": sd, "focus_tickers": focus_tickers},
                   best_path)
    else:
        bad += 1
        if bad >= patience:
            print(f"Early stopping at epoch {ep}")
            break

# Restore best weights
if best[1] is not None:
    model.load_state_dict(best[1])
    print("Loaded best checkpoint (lowest val loss). Saved to:", best_path)


Epoch 01 | train 1.0861 | val 1.0910 | val-acc 0.377
   · Val IC: 0.0237 | Precision@10%: 0.531
   · focus val counts: {'TSLA': 239, 'AAPL': 239, 'NVDA': 239}
   · TSLA val-acc: 0.477 | P@10%: 0.565 (n=239)
   · AAPL val-acc: 0.414 | P@10%: 0.609 (n=239)
   · NVDA val-acc: 0.372 | P@10%: 0.478 (n=239)
Epoch 02 | train 1.0831 | val 1.0893 | val-acc 0.375
   · Val IC: 0.0236 | Precision@10%: 0.531
   · focus val counts: {'TSLA': 239, 'AAPL': 239, 'NVDA': 239}
   · TSLA val-acc: 0.485 | P@10%: 0.522 (n=239)
   · AAPL val-acc: 0.393 | P@10%: 0.609 (n=239)
   · NVDA val-acc: 0.385 | P@10%: 0.522 (n=239)


KeyboardInterrupt: 

In [13]:
# ==========================================================
# Validation Backtest Simulator (per-ticker; base/global model)
# - Strategy A: long-only when score in top q per ticker
# - Strategy B: long-short (top q long, bottom q short) per ticker
# - Optional isotonic calibration on TRAIN split (no leakage)
# - Transaction costs in bps (round-trip modeled on entry/exit/flip)
# Metrics: total return, CAGR* (approx), Sharpe, max drawdown
# ==========================================================
import os, pickle, numpy as np, torch, torch.nn as nn
from torch.utils.data import DataLoader
from sklearn.isotonic import IsotonicRegression

# ---------- (A) Load preprocessed data if needed ----------
# Comment this block if X_all etc. are already in memory
DATA_PATH  = "/content/gdrive/MyDrive/sp500_quantformer_preprocessed.pkl"
with open(DATA_PATH, "rb") as f:
    data_bundle = pickle.load(f)
X_all = data_bundle["X_all"].astype(np.float32)
y_all = data_bundle["y_all"].astype(np.float32)          # continuous returns
sample_tickers = data_bundle["sample_tickers"]
mu = data_bundle.get("mu", None)
sd = data_bundle.get("sd", None)

# Normalize exactly like training
if mu is not None and sd is not None:
    X_used = (X_all - mu) / (sd + 1e-9)
else:
    mu = X_all.mean(axis=0, keepdims=True)
    sd = X_all.std(axis=0, keepdims=True) + 1e-9
    X_used = (X_all - mu) / sd

# ---------- (B) Model definition + load base checkpoint ----------
class Quantformer(nn.Module):
    def __init__(self, in_dim, num_bins, d_model=192, nhead=12, num_layers=6, dim_ff=384, dropout=0.25):
        super().__init__()
        self.embed = nn.Linear(in_dim, d_model)
        layer = nn.TransformerEncoderLayer(
            d_model=d_model, nhead=nhead, dim_feedforward=dim_ff,
            batch_first=True, norm_first=True, dropout=dropout, activation="gelu"
        )
        self.encoder = nn.TransformerEncoder(layer, num_layers)
        self.head = nn.Sequential(nn.LayerNorm(d_model), nn.Linear(d_model, num_bins))
    def forward(self, x):
        z = self.encoder(self.embed(x))
        return self.head(z.mean(1))

def load_base_model(ckpt_path, in_dim, num_bins, device):
    # PyTorch 2.6 safe-loading shim
    from contextlib import suppress
    with suppress(Exception):
        import numpy as _np
        torch.serialization.add_safe_globals([_np._core.multiarray._reconstruct])
    try:
        ckpt = torch.load(ckpt_path, map_location=device, weights_only=True)
    except Exception:
        ckpt = torch.load(ckpt_path, map_location=device, weights_only=False)
    state = ckpt["state_dict"] if isinstance(ckpt, dict) and "state_dict" in ckpt else ckpt
    model = Quantformer(in_dim, num_bins)
    model.load_state_dict(state, strict=False)
    return model

CKPT_PATH = "/content/gdrive/MyDrive/quantformer_best.pth"   # <- global/base checkpoint
device = "cuda" if torch.cuda.is_available() else "cpu"

# ---------- (C) Helper: build global 3-class labels like training ----------
def label_three_bins(y):
    q = np.quantile(y, [1/3, 2/3])
    return np.digitize(y, q, right=True).astype(np.int64)
y_cls = label_three_bins(y_all)
NUM_BINS = int(y_cls.max()) + 1
FEATS = X_used.shape[2]

# ---------- (D) Per-ticker chrono split (last 20% = validation) ----------
def chrono_split_indices(tickers, target, val_frac=0.2):
    t = np.asarray(tickers).astype(str)
    m = (np.char.upper(np.char.strip(t)) == target.upper().strip())
    idx = np.where(m)[0]
    n = len(idx)
    cut = max(1, int((1 - val_frac) * n))
    return idx[:cut], idx[cut:]  # train_idx, val_idx

# ---------- (E) Get model scores on arrays ----------
@torch.no_grad()
def predict_probs(model, X):
    model.eval()
    bs = 1024
    out = []
    X_t = torch.tensor(X, dtype=torch.float32, device=device)
    for i in range(0, len(X_t), bs):
        logits = model(X_t[i:i+bs])
        out.append(torch.softmax(logits, dim=1).cpu().numpy())
    return np.concatenate(out, axis=0)  # (N,3)

# ---------- (F) Infer which class is UP/DOWN by mean return where class wins ----------
def infer_up_down_ids(probs, y_cont):
    pred = probs.argmax(1)
    means = []
    for c in range(probs.shape[1]):
        m = (pred == c)
        means.append((c, y_cont[m].mean() if m.any() else -1e9))
    means.sort(key=lambda x: x[1])
    down_id = means[0][0]
    up_id   = means[-1][0]
    return int(up_id), int(down_id)

# ---------- (G) Backtest core (per ticker) ----------
def backtest_per_ticker(returns, score, mode="long_only", top_q=0.10, cost_bps=10):
    """
    returns: (N,) simple daily returns for the ticker (validation segment)
    score:   (N,) trading score aligned to returns (higher=more bullish)
    mode:    "long_only" or "long_short"
    top_q:   quantile for entries (e.g., 0.10 => top 10%)
    cost_bps: per trade side, e.g., 10 bps (0.001). Round-trip applied on entry/exit/flip.
    """
    N = len(returns)
    thr_long = np.quantile(score, 1 - top_q)
    if mode == "long_only":
        pos = (score >= thr_long).astype(float)  # 1 or 0
    elif mode == "long_short":
        thr_short = np.quantile(score, top_q)
        pos = np.zeros(N, dtype=float)
        pos[score >= thr_long] = 1.0
        pos[score <= thr_short] = -1.0
    else:
        raise ValueError("mode must be 'long_only' or 'long_short'")

    # Trades where position changes → apply costs
    pos_prev = np.roll(pos, 1); pos_prev[0] = 0.0
    trades = np.abs(pos - pos_prev)  # 1 on enter/exit, 2 on flip (-1->+1 or +1->-1)
    # round-trip cost per change side:
    # If you assume cost per side (enter or exit) = cost_bps, then cost per flip is 2*cost_bps
    costs = trades * (cost_bps / 1e4)  # cost as fraction of notional

    # PnL: position * return minus transaction costs (applied on change days)
    pnl = pos * returns - costs
    equity = (1 + pnl).cumprod()
    total_ret = equity[-1] - 1.0

    # Metrics
    avg_daily = pnl.mean()
    vol_daily = pnl.std(ddof=1) + 1e-12
    sharpe = np.sqrt(252) * avg_daily / vol_daily  # daily → annualized
    # Max drawdown
    roll_max = np.maximum.accumulate(equity)
    drawdown = equity / roll_max - 1.0
    mdd = drawdown.min()

    # Approx CAGR (no dates; assume 252 trading days/year; len=N)
    years = max(len(pnl) / 252.0, 1e-9)
    cagr = equity[-1] ** (1.0 / years) - 1.0

    return {
        "equity": equity,
        "pnl": pnl,
        "total_return": float(total_ret),
        "cagr": float(cagr),
        "sharpe": float(sharpe),
        "max_drawdown": float(mdd),
        "trades": int((trades > 0).sum())
    }

# ---------- (H) Run the simulation ----------
def simulate_validation(
    tickers=("TSLA","AAPL","NVDA"),
    ckpt_path=CKPT_PATH,
    mode="long_only",           # "long_only" or "long_short"
    top_q=0.10,                 # top quantile for entries
    cost_bps=10,                # per-side costs in bps
    use_isotonic=True           # calibrate score on TRAIN, apply on VAL
):
    model = load_base_model(ckpt_path, FEATS, NUM_BINS, device).to(device)
    results = {}
    total_pnl = []

    for tkr in tickers:
        tr_idx, va_idx = chrono_split_indices(sample_tickers, tkr, val_frac=0.2)
        if len(va_idx) == 0:
            print(f"{tkr}: no validation data; skipping.")
            continue

        Xtr, ytr_cont = X_used[tr_idx], y_all[tr_idx]
        Xva, yva_cont = X_used[va_idx], y_all[va_idx]

        # Model probabilities
        probs_tr = predict_probs(model, Xtr)
        probs_va = predict_probs(model, Xva)

        # Infer UP/DOWN ids on TRAIN; score = margin on VAL
        up_id, down_id = infer_up_down_ids(probs_tr, ytr_cont)
        margin_tr = probs_tr[:, up_id] - probs_tr[:, down_id]
        margin_va = probs_va[:, up_id] - probs_va[:, down_id]

        # Optional isotonic calibration (fit on TRAIN only; apply to VAL)
        if use_isotonic:
            order = np.argsort(margin_tr)
            ir = IsotonicRegression(out_of_bounds="clip")
            ir.fit(margin_tr[order], ytr_cont[order])
            score_va = ir.predict(margin_va)
        else:
            score_va = margin_va

        # Backtest per ticker
        bt = backtest_per_ticker(
            returns=yva_cont, score=score_va,
            mode=mode, top_q=top_q, cost_bps=cost_bps
        )
        results[tkr] = bt
        total_pnl.append(bt["pnl"])

        print(f"[{tkr}] mode={mode}, top_q={top_q}, cost={cost_bps}bps  "
              f"Ret={bt['total_return']:.2%}  CAGR≈{bt['cagr']:.2%}  "
              f"Sharpe={bt['sharpe']:.2f}  MDD={bt['max_drawdown']:.2%}  Trades={bt['trades']}")

    # Aggregate across tickers (equal notional per ticker)
    if total_pnl:
        pnl_all = np.vstack(total_pnl).mean(axis=0)  # equal-weight across tickers
        equity = (1 + pnl_all).cumprod()
        total_ret = equity[-1] - 1.0
        avg_daily = pnl_all.mean()
        vol_daily = pnl_all.std(ddof=1) + 1e-12
        sharpe = np.sqrt(252) * avg_daily / vol_daily
        roll_max = np.maximum.accumulate(equity)
        mdd = (equity / roll_max - 1.0).min()
        years = max(len(pnl_all) / 252.0, 1e-9)
        cagr = equity[-1] ** (1.0 / years) - 1.0

        print(f"\n[AGG] Equal-weighted across tickers → "
              f"Ret={total_ret:.2%}  CAGR≈{cagr:.2%}  Sharpe={sharpe:.2f}  MDD={mdd:.2%}")
        results["_AGG_"] = {
            "total_return": float(total_ret),
            "cagr": float(cagr),
            "sharpe": float(sharpe),
            "max_drawdown": float(mdd),
            "equity": equity,
            "pnl": pnl_all
        }
    return results

# ---------- (I) Run some scenarios ----------
# 1) Long-only, top 10%, 10 bps costs
res1 = simulate_validation(
    tickers=("TSLA","AAPL","NVDA"),
    ckpt_path=CKPT_PATH,
    mode="long_only",
    top_q=0.10,
    cost_bps=10,
    use_isotonic=True
)

# 2) Long-short, 10% tails, 10 bps costs
res2 = simulate_validation(
    tickers=("TSLA","AAPL","NVDA"),
    ckpt_path=CKPT_PATH,
    mode="long_short",
    top_q=0.10,
    cost_bps=10,
    use_isotonic=True
)


/usr/local/lib/python3.12/dist-packages/torch/nn/modules/transformer.py:392: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.norm_first was True
  warnings.warn(


[TSLA] mode=long_only, top_q=0.1, cost=10bps  Ret=127.00%  CAGR≈137.36%  Sharpe=2.51  MDD=-11.53%  Trades=11
[AAPL] mode=long_only, top_q=0.1, cost=10bps  Ret=16.88%  CAGR≈17.88%  Sharpe=0.93  MDD=-18.65%  Trades=3
[NVDA] mode=long_only, top_q=0.1, cost=10bps  Ret=13.22%  CAGR≈13.99%  Sharpe=0.52  MDD=-41.14%  Trades=5

[AGG] Equal-weighted across tickers → Ret=50.20%  CAGR≈53.56%  Sharpe=2.00  MDD=-17.82%


/usr/local/lib/python3.12/dist-packages/torch/nn/modules/transformer.py:392: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.norm_first was True
  warnings.warn(


[TSLA] mode=long_short, top_q=0.1, cost=10bps  Ret=182.86%  CAGR≈199.32%  Sharpe=1.96  MDD=-43.43%  Trades=12
[AAPL] mode=long_short, top_q=0.1, cost=10bps  Ret=18.57%  CAGR≈19.67%  Sharpe=0.71  MDD=-22.89%  Trades=3
[NVDA] mode=long_short, top_q=0.1, cost=10bps  Ret=4.88%  CAGR≈5.16%  Sharpe=0.35  MDD=-41.14%  Trades=5

[AGG] Equal-weighted across tickers → Ret=68.15%  CAGR≈72.97%  Sharpe=2.31  MDD=-11.36%
